<a href="https://colab.research.google.com/github/GiuliaMaragno/Progetto-Sii-ML/blob/main/EfficientNetB0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from matplotlib import pyplot as plt




from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [2]:
train_path ='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Train'
valid_path='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Valid'
test_path='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Test'

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.1, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_set = train_datagen.flow_from_directory(directory=train_path, target_size=(224, 224), class_mode='categorical', batch_size=100)
valid_set = valid_datagen.flow_from_directory(directory=valid_path, target_size=(224, 224), class_mode='categorical', batch_size=100)
test_set = valid_datagen.flow_from_directory(directory=test_path, target_size=(224,224), class_mode='categorical', batch_size=1, shuffle=False)

Found 5646 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [4]:
model=tf.keras.applications.EfficientNetB0()

21834768/21834768 [==============================] - 0s 0us/step


In [5]:
x= model.output
x = Flatten()(x)
x= Dense(units=8, activation='softmax')(x)
model= Model(inputs=model.input, outputs=x)

In [6]:
for layer in model.layers[:-30]:
    layer.trainable = False

In [7]:
from tensorflow.python.keras import optimizers

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, weight_decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
step_size_train=train_set.n//train_set.batch_size
step_size_valid=valid_set.n//valid_set.batch_size

In [9]:
model.fit(x=train_set, steps_per_epoch=step_size_train, validation_data= valid_set, validation_steps=step_size_valid, epochs=30)

Epoch 1/30
56/56 [==============================] - 1712s 30s/step - loss: 2.0797 - accuracy: 0.1217 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 2/30
56/56 [==============================] - 189s 3s/step - loss: 2.0798 - accuracy: 0.1269 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 3/30
56/56 [==============================] - 191s 3s/step - loss: 2.0797 - accuracy: 0.1275 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 4/30
56/56 [==============================] - 189s 3s/step - loss: 2.0797 - accuracy: 0.1131 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 5/30
56/56 [==============================] - 189s 3s/step - loss: 2.0797 - accuracy: 0.1232 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 6/30
56/56 [==============================] - 189s 3s/step - loss: 2.0798 - accuracy: 0.1268 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 7/30
56/56 [==============================] - 188s 3s/step - loss: 2.0798 - accuracy: 0.1210 - val_loss: 2.0795 - val_accuracy: 0.1250
Epoch 8/30


In [10]:
model.save('/content/drive/MyDrive/Progetto/Models/Modello EfficientNetB0 2')

TypeError: ignored

In [11]:
pred = model.predict(test_set, steps = len(test_set), verbose = 2)

800/800 - 216s - 216s/epoch - 269ms/step


In [12]:
predicted_class = np.argmax(pred, axis = 1)
true_classes = test_set.classes
class_labels= list(test_set.class_indices.keys())

from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)

                        precision    recall  f1-score   support

    dyed-lifted-polyps       0.00      0.00      0.00       100
dyed-resection-margins       0.12      1.00      0.22       100
           esophagitis       0.00      0.00      0.00       100
          normal-cecum       0.00      0.00      0.00       100
        normal-pylorus       0.00      0.00      0.00       100
         normal-z-line       0.00      0.00      0.00       100
                polyps       0.00      0.00      0.00       100
    ulcerative-colitis       0.00      0.00      0.00       100

              accuracy                           0.12       800
             macro avg       0.02      0.12      0.03       800
          weighted avg       0.02      0.12      0.03       800



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
